# 1. Defining records in yabadaba

This Notebook gives a overview of how yabadaba is designed, shows how Record classes can be defined, and shows off some Record features.

In [45]:
import yabadaba

## Core design of yabadaba

The core design of yabadaba centers around 4 categories of classes

- __Database__ The Database classes manage database interactions and provide a (nearly) uniform interface for interacting with different database infrastructures.  Note that the underlying database-specific APIs are still accessible should there be a missing feature.
   
- __Record__ The Record classes are designed to be the Python representations of your data. Each "Record" is for a single data schema, serves as the primary object for interacting with that data, and contains methods for transforming the data between different formats used by the different database infrastructures.

- __Value__ The Value classes each represent a different basic data type.  Defining Record attributes as Value objects allows for the associated data to be automatically transformed between different representations as needed for the database interactions. 

- __Query__ The Query classes define a generic database query operation and how to implement that operation in an efficient way for the different Database types. Most of the Value objects have a single default Query already assigned to them.

Classes of all of these categories are handled in a completely modular manner and have an associated "manager" object.  If you want to check the status of the different styles, you can call the check_styles() method of the associated manager object, or call yabadaba.check_modules() to see all.

In [2]:
yabadaba.check_modules()

Database styles that passed import:
- local: <class 'yabadaba.database.LocalDatabase.LocalDatabase'>
- mongo: <class 'yabadaba.database.MongoDatabase.MongoDatabase'>
- cdcs: <class 'yabadaba.database.CDCSDatabase.CDCSDatabase'>
Database styles that failed import:


Record styles that passed import:
Record styles that failed import:


Query styles that passed import:
- str_contains: <class 'yabadaba.query.StrContainsQuery.StrContainsQuery'>
- str_match: <class 'yabadaba.query.StrMatchQuery.StrMatchQuery'>
- list_contains: <class 'yabadaba.query.ListContainsQuery.ListContainsQuery'>
- int_match: <class 'yabadaba.query.IntMatchQuery.IntMatchQuery'>
- float_match: <class 'yabadaba.query.FloatMatchQuery.FloatMatchQuery'>
- date_match: <class 'yabadaba.query.DateMatchQuery.DateMatchQuery'>
Query styles that failed import:


Value styles that passed import:
- base: <class 'yabadaba.value.Value.Value'>
- str: <class 'yabadaba.value.StrValue.StrValue'>
- longstr: <class 'yabadaba.value.LongStrV

In [31]:
yabadaba.recordmanager.check_styles()

Record styles that passed import:
Record styles that failed import:



Note that there are no Records at the moment!  That's because this is the base package and knows nothing about *your* data.

## Record example: FAQ

### Defining a Record class

Let's design a simple Record class for a FAQ that has only two fields: a "question" and an "answer".

In [3]:
from yabadaba.record import Record
from yabadaba import load_value

class FAQ(Record):
    """
    Class for representing FAQ (frequently asked question) records.
    """

    ########################## Basic metadata fields ##########################

    @property
    def style(self):
        """str: The record style"""
        return 'faq'

    @property
    def modelroot(self):
        """str: The root element of the content"""
        return 'faq'
    
    @property
    def xsl_filename(self):
        """tuple: The module path and file name of the record's xsl html transformer"""
        return ('yabadaba_demo.record.faq', 'FAQ.xsl')

    @property
    def xsd_filename(self):
        """tuple: The module path and file name of the record's xsd schema"""
        return ('yabadaba_demo.record.faq', 'FAQ.xsd')

    ####################### Define Values and attributes #######################

    def _init_value_objects(self) -> list:
        """
        Method that defines the value objects for the Record.  This should
        1. Call the method's super() to get default Value objects.
        2. Use yabadaba.load_value() to build Value objects that are set to
           private attributes of self.
        3. Append the list returned by the super() with the new Value objects.

        Returns
        -------
        value_objects: A list of all value objects.
        """
        value_objects = super()._init_value_objects()
        
        self.__question = load_value('longstr', 'question', self)
        value_objects.append(self.__question)

        self.__answer = load_value('longstr', 'answer', self)
        value_objects.append(self.__answer)

        return value_objects

    @property
    def question(self):
        """str: The frequently asked question."""
        return self.__question.value

    @question.setter
    def question(self, val):
        self.__question.value = val

    @property
    def answer(self):
        """str: The answer to the frequently asked question."""
        return self.__answer.value

    @answer.setter
    def answer(self, val):
        self.__answer.value = val


What all was done to define the FAQ Record?

1. The base yabadaba Record class was imported and FAQ was made a subclass of it.
2. A style attribute was defined that gives a simple style name to the Record subclass.  This is used to differentiate between the different Record classes.
3. A modelroot attribute was defined that gives the name for the root element of the JSON/XML representation of the data.  This is typically derived from the style/class name and helps differentiate the different types of data when stored as JSON or XML.
4. An _init_value_objects method was defined.  In it, new Value objects are defined using the yabadaba.load_value method for both question and answer and these Value objects are saved as __private variables.  The value_objects list is appended with the new objects and is returned.
5. Class properties and setters are defined for question and answer where they point to the "value" attribute of the Value objects.

### Integrating the Record into yabadaba

The final step in integrating the record into yabadaba is to add it to the recordmanager.  Since we defined the FAQ class in this notebook, we can simply add it to the recordmanager.loaded_styles dict.

In [32]:
yabadaba.recordmanager.loaded_styles['faq'] = FAQ
yabadaba.recordmanager.check_styles()

Record styles that passed import:
- faq: <class '__main__.FAQ'>
Record styles that failed import:



This works great if you define or import a Record subclass directly before adding it to the manager.  You can also define a Record subclass in a separate module and have that module automatically add it to the recordmanager by calling recordmanager.import_style().  More details are given on this in the next Notebook.

Also, note that if you wish new subclasses of Database, Value, and Query can be added in a similar way using the corresponding manager objects in yabadaba.

## Basic Record features

And now for the cool stuff!

Let's define a FAQ object, then assign question and answer values.  Since we defined FAQ in this Notebook, we can init it directly or we can use the yabadaba.load_record() method and give the 'faq' record style.

As a quick note, we also give the record a name.  Think of this as the file name for the record should it be saved to a database.

In [33]:
faq1 = yabadaba.load_record('faq', name='test1')

Now, we can set and get the question and answer values using the associated class properties.

In [34]:
faq1.question = "How many roads must a man walk down?"
faq1.question

'How many roads must a man walk down?'

In [35]:
faq1.answer = 42
faq1.answer

'42'

Note that the answer was given as an int but is now a str! This is because answer was defined as a 'longstr' Value in the record.  The Value objects handle automatic data type conversions to what they are meant to represent!

We can alternatively provide question and answer values when defining a FAQ.  The Record automatically knows to assign the given values.

In [36]:
faq2 = yabadaba.load_record('faq', name='test2', question="Who's on first?", answer='Yes.')
print(faq2.question)
print(faq2.answer)

Who's on first?
Yes.


## Data format conversions

The Record objects also have a number of methods for converting the data between different formats.

### Flat metadata dict

metadata generates a flat dictionary for the record that includes any simple data type values (int, str, float, bool, None) 

In [37]:
faq1.metadata()

{'name': 'test1',
 'question': 'How many roads must a man walk down?',
 'answer': '42'}

### Tree-like JSON/XML data models

JSON/XML conversions are managed by model, build_model(), load_model(), and reload_model().

- model is embedded DataModelDicts representing the tree-like data structure of JSON/XML.  DataModelDict simply expands upon the basic dict with JSON and XML converters and additional tools that help with such data.  This only exists *after* loading a model or calling build_model().
- build_model() creates model based on the current values of the Record class attributes.
- load_model() reads in JSON or XML content, sets model, and sets the values of the Record class attributes.
- reload_model() updates the values of the Record class attributes based on what is currently in model.  This is useful if you alter the data in the model representation.

The important thing to keep in mind is that the "model" representation is separate from the "attribute" representation.  

Calling build_model() constructs the model and returns its value.

In [38]:
faq1.build_model()

DataModelDict([('faq',
                DataModelDict([('question',
                                'How many roads must a man walk down?'),
                               ('answer', '42')]))])

model can be directly transformed into JSON or XML using the associated methods.

In [39]:
print(faq1.model.json(indent=2))

{
  "faq": {
    "question": "How many roads must a man walk down?",
    "answer": "42"
  }
}


In [40]:
print(faq1.model.xml(indent=2))

<?xml version="1.0" encoding="utf-8"?>
<faq>
  <question>How many roads must a man walk down?</question>
  <answer>42</answer>
</faq>


If you want, you can interact with the data in the model representation directly to retrieve or update values.  *But* remember that changing values in model will not change the associated class attributes unless you call reload_model()!

In [28]:
faq1.model['faq']['answer']

'42'

In [29]:
faq1.model['faq']['answer'] = '8675309'
print(faq1.answer)

faq1.reload_model()
print(faq1.answer)

42
8675309


### Loading content

You can load in JSON or XML content using the load_model() method or by using the model parameter when initializing a Record.  The JSON/XML content can be given as a string, a file path, or an open file object.

In [41]:
faqjson = """{
    "faq": {
        "question": "Fuzzywuzzy was a bear. Fuzzywuzzy had no hair. So Fuzzywuzzy wasn't fuzzy, was he?",
        "answer": "Nope."
    }
}"""
faq3 = yabadaba.load_record('faq', name='fuzzy', model=faqjson)
faq3.metadata()

{'name': 'fuzzy',
 'question': "Fuzzywuzzy was a bear. Fuzzywuzzy had no hair. So Fuzzywuzzy wasn't fuzzy, was he?",
 'answer': 'Nope.'}

Since you are loading the model contents in, the Record.model attribute will exist without needing to call build_model().

In [44]:
print(faq3.model.json(indent=2))

{
  "faq": {
    "question": "Fuzzywuzzy was a bear. Fuzzywuzzy had no hair. So Fuzzywuzzy wasn't fuzzy, was he?",
    "answer": "Nope."
  }
}
